<a href="https://colab.research.google.com/github/anjunjin/PFE-ING3-IA/blob/branch1/GetCsvFromJsonOnline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import urllib.request, json 
with urllib.request.urlopen("https://www.lemonde.fr/webservice/decodex/updates") as url: # Obtenir le fichier JSON de https://www.lemonde.fr/webservice/decodex/updates
    data = json.loads(url.read().decode())

# data.keys() #On obtient le premier dictionnaire, sites et url, contient des dictionnaire
# data["sites"].keys() #On obtient le sous-dictionnaire, chaque site, contient des tableaux

In [ ]:
# print(data)
# data.keys() # Vérifie la forme que prend les fichier JSON
# data["sites"]
# data["sites"]['1980']
# data["sites"]['1980'][2]
# data["sites"].keys()
# for i in data["sites"].keys():
#   print(type(i))

# for i in data["sites"].keys():
#   print(data["sites"][i][2])

# data["urls"].keys()
# data["urls"]['the-postillon.com']

In [ ]:
import pandas as pd

dataset = []

for i in data["sites"].keys(): # On créé un tableau avec la liste de tous les sites du fichier JSON. On a leur identifiant, leur niveau de fiabilité, une description, et 2 noms dont l'un est simplifié
  dataset.append([i,data["sites"][i][0], data["sites"][i][1], data["sites"][i][2], data["sites"][i][3]])

tableauSites = pd.DataFrame(dataset, columns=['Identifiant','Fiabilité','Description','Nom1','Nom2']) #Pour que le tableau prenne la forme d'un tableau (dataset n'étant en réalité qu'une liste)
tableauSites

,Identifiant,Fiabilité,Description,Nom1,Nom2
0,1982,2,France-Soir était un journal fondé par Pierre ...,France Soir,france-soir
1,1981,2,"Thierry Casasnovas, vidéaste et animateur de l...",Régénère (Thierry Casasnovas),regenere-thierry-casasnovas
2,1980,2,Silvano Trotta est le créateur d’une société s...,Silvano Trotta,silvano-trotta
3,1979,2,"Jasper Mader est une chaîne YouTube, animée pa...",Jasper Mader,jasper-mader
4,1978,2,Une webtélé suisse créée notamment par Chloé F...,Agora TV,agora-tv
...,...,...,...,...,...
1338,577,4,Dailymotion n'est pas une source d'information...,Dailymotion,dailymotion
1339,576,4,Une plateforme participative qui n'est pas ori...,Agoravox,agoravox
1340,575,4,9gag n'est pas une source d'information en soi...,9gag,9gag
1341,574,4,L'encyclopédie collaborative en ligne compile ...,Wikipédia,wikipedia


In [ ]:
# tableauSites['Twitter'] = tableauSites['Nom1']
# tableauSites

import re

# data["urls"] = dict(sorted(data["urls"].items(), key=lambda item: item[1], reverse=True))

# dataset = []
# for i in data["urls"].keys(): # On créé un tableau contenant un lien twitter pour chaque site faisant parti du fichier JSON, si le fichier ne contenaint pas de lien twitter, on le remplace par Nan (objectif serait de trouver le lien avec une sorte de fonction TwitterSearchAccount)
#   if (dataset != []):
#     if ((data["urls"][i] - dataset[len(dataset)-1][0] < -1) or (data["urls"][i] - dataset[len(dataset)-1][0] > 1)): # Cas où pour un identifiant particulié aucun lien twitter n'a été trouvé dans le fichier JSON
#       dataset.append([data["urls"][i]+1, "NaN"]) # On rajoute alors une ligne NaN pour cet identifiant (objectif étant de directement chercher le compte twitter avec une fonction de recherche de compte)
#   if re.search('twitter', i): # Y a un lien twitter, on l'ajoute à notre tableau
#     dataset.append([data["urls"][i], i])

# tableauSites['Twitter'] = pd.DataFrame([i[1] for i in dataset])
# #Oh non le JSON est pas trié !
# tableauSites

# print(data["urls"])

dt = [] #Pour déterminer toutes les lignes dans le JSON qui ne sont pas des liens twitter
compteur = 0
for i in data["urls"].keys():
  if not(re.search('twitter', i)):
    dt.append(compteur)
  compteur = compteur+1

df = pd.DataFrame([data["urls"][i], i] for i in data["urls"].keys()) #On met tous les liens dans df
df = df.sort_index() #On trie car le JSON n'est pas trié
df = df.drop(index=dt) #On retire les lignes ne contenant pas de lien twitter, on utilise dt pour ça
df = df.reset_index() #On retrouve un idexage normal
df = df.drop(columns="index") #On retire la colonne qui s'est rajoutée à cause du reset_index()

#Liste de tous les identifiants qui n'ont pas de lien twitter
j = len(df.index)
dd = [] #Pour rajouter des lignes NaN aux domaines sans lien twitter

for i in range(j-1): #On créé un tableau avec tous les domaines sans lien twitter, tableau de forme [identifiant, "NaN"]
  for k in range(1, df[0][i] - df[0][i+1]): #Parfois plusieurs domaines à la suite n'ont pas de lien twitter, il faut rajouter une ligne pour chacun d'entre eux
    dd.append([df[0][i]-k, "NaN"]) # Cas où pour un identifiant particulié aucun lien twitter n'a été trouvé dans le fichier JSON

#Liste des tous les identifiants qui n'existent pas
dt = [] #Pour déterminer toutes les lignes dans le JSON font partis de "sites" mais qui n'ont pas de lien twitter
compteur = 0
j = 1982
for i in data["sites"].keys():
  for k in range(int(i)+1, j):
    dt.append([k, "NaN"])
  j = int(i)

for i in dt: #On retire de la liste des identifiants sans lien twitter les identifiants qui n'existent pas
  dd.remove(i)

df = df.append(pd.DataFrame(dd)) #On rajoute les nouvelles lignes
df = df.sort_values([0], ascending=False)
df = df.reset_index()
df = df.drop(columns="index")

# dt = [] #Pour déterminer toutes les lignes dans le JSON qui n'ont pas d'identifiant, comme 1944
# compteur = 0
# j = 1982
# for i in data["sites"].keys():
#   for k in range(int(i)+1, j):
#     dt.append([k, "NaN"])
#   j = int(i)

tableauSites['Twitter'] = df[1]
tableauSites

,Identifiant,Fiabilité,Description,Nom1,Nom2,Twitter
0,1982,2,France-Soir était un journal fondé par Pierre ...,France Soir,france-soir,twitter.com/france_soir
1,1981,2,"Thierry Casasnovas, vidéaste et animateur de l...",Régénère (Thierry Casasnovas),regenere-thierry-casasnovas,twitter.com/ThierryRegenere
2,1980,2,Silvano Trotta est le créateur d’une société s...,Silvano Trotta,silvano-trotta,twitter.com/silvano_trotta
3,1979,2,"Jasper Mader est une chaîne YouTube, animée pa...",Jasper Mader,jasper-mader,NaN
4,1978,2,Une webtélé suisse créée notamment par Chloé F...,Agora TV,agora-tv,NaN
...,...,...,...,...,...,...
1338,577,4,Dailymotion n'est pas une source d'information...,Dailymotion,dailymotion,twitter.com/Fakir_
1339,576,4,Une plateforme participative qui n'est pas ori...,Agoravox,agoravox,NaN
1340,575,4,9gag n'est pas une source d'information en soi...,9gag,9gag,twitter.com/PaulJorion
1341,574,4,L'encyclopédie collaborative en ligne compile ...,Wikipédia,wikipedia,twitter.com/cahiersdufoot


In [ ]:
tableauSites.to_csv('json1.csv')
!cp json1.csv "/content/drive/My Drive/PFE"
df.to_csv('json2.csv')
!cp json2.csv "/content/drive/My Drive/PFE"

In [ ]:
# 1 : site parodique
# 2 : non fiable
# 3 : site à la fiabilité douteuse
# 4 : fiable